In [8]:
%pylab inline
import numpy as np
import pandas as pd
data = []
datapath = '/home/bigdatas16/SGX-OrderBook-Tick-Data-Trading-Strategy-/CN_Futures_2014.01.24.csv'
data = pd.read_csv(datapath)
#data_CNF14 = data[data.Series == 'CNF14']
#data = data_CNF14
data.head()

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


,Series,SequenceNumber,TimeStamp,OrderNumber,OrderBookPosition,Price,QuantityDifference,Trade,BidOrAsk,BestPrice,BestQuantity
0,CNF14,1,2014-01-24D04:21:49.419665000,C791DA0F799A5E9A,1,710000,2,,A,677000,19
1,CNF14,2,2014-01-24D04:21:49.419665000,E1C9F25394A679A6,2,719500,1,,A,677000,19
2,CNF14,3,2014-01-24D04:21:49.419665000,C02B6F481F080128,1,684000,100,,A,677000,19
3,CNF14,4,2014-01-24D04:21:49.419665000,C55C20A9615D40FD,2,692500,1,,A,677000,19
4,CNF14,5,2014-01-24D04:21:49.419665000,E5D428BDEDEDCCCD,1,678500,20,,A,677000,19


In [15]:
data_CNF = data[data.Series == data.Series[0]]
data = data_CNF
len(data)

143284

In [10]:
import time 
def insert(order_book_data,data_to_insert,ob_position):
    top = order_book_data[0:ob_position]
    bottom = order_book_data[ob_position:]
    return pd.concat((top,data_to_insert,bottom)).reset_index(drop = True)

start  =  time.time()

order_book_bid = []
order_book_ask = []
timestamp = data.TimeStamp[0]
temp_ask = 0
temp_bid = 0
bid = []
ask = []
timestamp_ = []
x1 = data[(data.BidOrAsk == 'A')].TimeStamp.unique()
x2 = data[(data.BidOrAsk == 'B')].TimeStamp.unique()
y = data[data['TimeStamp'].str.contains(timestamp)]
bid.append(y[(y.BidOrAsk == 'B')][["Price","OrderNumber","QuantityDifference"]]) # bid
ask.append(y[(y.BidOrAsk == 'A')][["Price","OrderNumber","QuantityDifference"]]) # ask
a = bid[0].sort(['Price'],ascending = [False])
b = ask[0].sort(['Price'],ascending = [True])    
order_book_bid.append(a[a.QuantityDifference != 0].reset_index(drop = True))
order_book_ask.append(b[b.QuantityDifference != 0].reset_index(drop = True))

order_book_bid.append(0)
order_book_ask.append(0)
#a = []
#b = []

for i in range(len(y),3200,1):
    print '------------------------------------------------------------------------------'
    print i
    #print data.BidOrAsk[i] == 'A'
    if data.BidOrAsk[i] == 'A':

        if int(data[['QuantityDifference']][i:i+1].values) > 0 :
            
            if data.TimeStamp[i] == x1[temp_ask]:
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1

                order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)

                if position_ == 0 and len(order_book_ask[0]) > 1:
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error1 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
                elif 0 < position_< (len(order_book_ask[temp_ask])-1):
                    
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:#or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error2 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'   
                        continue
                elif position_ == len(order_book_ask[temp_ask])-1:
                        
                    if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:# or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error3 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                
                        continue
                elif position_ == 0 and len(order_book_ask[0]) == 1:
                    print 'position and Best Price is True' 
                
            elif data.TimeStamp[i] != x1[temp_ask]:
                ###a.append(x1[temp_ask])
                #print order_book_ask[temp_ask]
                #print temp_ask
                if temp_ask == 0:
                    temp_ask = temp_ask + 1
                    order_book_ask[temp_ask] = order_book_ask[temp_ask - 1]
                    #print order_book_ask[temp_ask]
                    order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)                  
                    #print order_book_ask[temp_ask]
                    best_price = data[i:(i+1)]['BestPrice']
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                    #print 'position = %d'%(position_)
                    #print 'length-1 = %d'%(len(order_book_ask[temp_ask])-1)
                    #print order_book_ask[temp_ask][position_+1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]
                    #print order_book_ask[temp_ask][position_-1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]
                    #print order_book_ask[temp_ask]['Price'][0:1].iloc[0]# != data['BestPrice'][i]
                    if position_ == 0 and len(order_book_ask[0]) > 1:
                        if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error11 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            #print 'position and Best Price is True'                     
                            continue
                    elif 0 < position_< len(order_book_ask[temp_ask])-1:
                    
                        if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ -  1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error22 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_ask[temp_ask])-1:
                        
                        if order_book_ask[temp_ask][position_ -  1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:# or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error33 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            continue
                        else:
                            print 'position and Best Price is True'
                            continue
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True'
                        continue
                        
                    #print order_book_ask[temp_ask]                        
                else:
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1

                    order_book_ask.append(order_book_ask[temp_ask])
                    temp_ask = temp_ask + 1

                    order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)

                        
                    if position_ == 0:
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            print 'Some error111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                     
                            continue
                    elif 0 < position_< len(order_book_ask[temp_ask])-1:
                    
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            print 'Some error222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_ask[temp_ask])-1:
                        
                        if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                            print 'Some error333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                        
                            continue
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True'
                        continue                            
                            
                            
                            
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:    

            if data.TimeStamp[i] == x1[temp_ask]: 
                #print temp_ask
                #print order_book_ask[temp_ask]
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                
                if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                #print order_book_ask[temp_ask]

                if position_ == 0 and len(order_book_ask[0]) > 1:
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error1111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
                elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error2222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'  
                        continue
                elif position_ == len(order_book_ask[temp_ask])-1:
                        
                    if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                        print 'Some error3333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        continue                        
                    else:
                        print 'position and Best Price is True'                        
                        continue
                elif position_ == 0 and len(order_book_ask[0]) == 1:
                    print 'position and Best Price is True'
                    continue

            elif data.TimeStamp[i] != x1[temp_ask]:
                ###a.append(x1[temp_ask])
                #print temp_ask
                #print order_book_ask[temp_ask]
                order_book_ask.append(order_book_ask[temp_ask])
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                temp_ask = temp_ask + 1
                #print 'temp_ask2 = %d'%(temp_ask)
                if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                #print order_book_ask[temp_ask]
                
                if position_ == 0 and len(order_book_bid[0]) > 1:
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error11111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
                elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error22222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'  
                        continue
                elif position_ == len(order_book_ask[temp_ask])-1:
                        
                    if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                        print 'Some error33333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        continue                        
                    else:
                        print 'position and Best Price is True'                        
                        continue 
                elif position_ == 0 and len(order_book_ask[0]) == 1:
                    print 'position and Best Price is True'
                    continue                          

    elif data.BidOrAsk[i] == 'B':
        #print i
        #print int(data[['QuantityDifference']][i:i+1].values) > 0
        
        if int(data[['QuantityDifference']][i:i+1].values) > 0: #and len(order_book_bid[0]) > 0:
            print data.TimeStamp[i] == x2[temp_bid]
            
            if data.TimeStamp[i] == x2[temp_bid]:
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)
                #print position_
                
                if position_ == 0 and len(order_book_bid[0]) > 1:
                    if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error111111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
                        
                elif 0 < position_< len(order_book_bid[temp_bid])-1:
                    
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error222222 %d %d %d'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'   
                        continue
                elif position_ == len(order_book_bid[temp_bid])-1 and len(order_book_bid[0]) > 1:
                        
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:# or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error333333 %d %d %d'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                
                        continue
                elif position_ == 0 and len(order_book_bid[0]) == 1:
                    print 'position and Best Price is True'
                    continue
                
            elif data.TimeStamp[i] != x2[temp_bid]:
                ###b.append(x2[temp_bid])
                #print 'temp bid = %d'%(temp_bid)
                #print order_book_bid[temp_bid]
                if temp_bid == 0:
                    #print temp_bid
                    #print order_book_bid[temp_bid]
                    temp_bid = temp_bid + 1
                    order_book_bid[temp_bid] = order_book_bid[temp_bid - 1]
                    #print order_book_bid[temp_bid]
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)                  
                    #print order_book_bid[temp_bid]
                    best_price = data[i:(i+1)]['BestPrice']
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                    #print position_
                    #print len(order_book_bid[temp_bid]) - 1
                    
                    #print order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]
                    #print order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]
                    #print order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]
                    #print order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0]
                    #print order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0]
                    #print temp_bid
                    #print order_book_bid[temp_bid]
                    if position_ == 0  and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error1111111 %d %d %d'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                     
                            continue
                            
                    elif 0 < position_< len(order_book_bid[temp_bid])-1:
                    
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error2222222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_bid[temp_bid])-1:
                        
                        if order_book_bid[temp_bid][position_ -  1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error3333333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            continue
                        else:
                            print 'position and Best Price is True'
                            continue
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                            print 'position and Best Price is True'
                            continue
                else:
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                    order_book_bid.append(order_book_bid[temp_bid])
                    temp_bid = temp_bid + 1
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)
                    #print position_    
                    #print len(order_book_bid[temp_bid])-1      
                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error11111111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                     
                            continue
                    elif 0 < position_< len(order_book_bid[temp_bid])-1:
                    
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error22222222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_bid[temp_bid])-1:
                        
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error33333333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                        
                            continue
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                        print 'position and Best Price is True'
                        continue
                        
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:    

            if data.TimeStamp[i] == x2[temp_bid]: 
                #print temp_bid
                #print order_book_bid[temp_bid]
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                
                if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                #print order_book_bid[temp_bid]

                if position_ == 0 and len(order_book_bid[0]) > 1:
                    if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:# or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error19 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
                        
                elif 0 < position_< len(order_book_bid[temp_bid])-1:    
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error29 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'  
                        continue
                elif position_ == len(order_book_bid[temp_bid])-1:
                        
                    if position_ > 0 and order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                        print 'Some error39 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        continue
                    else:
                        print 'position and Best Price is True'                        
                        continue
                elif position_ == 0 and len(order_book_bid[0]) == 1:
                    print 'position and Best Price is True'
                    continue

            elif data.TimeStamp[i] != x2[temp_bid]:
                ###b.append(x2[temp_bid])
                #print temp_bid
                #print order_book_bid[temp_bid]
                order_book_bid.append(order_book_bid[temp_bid])
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                temp_bid = temp_bid + 1
                #print position_
                #print len(order_book_ask[temp_ask])-1
                #print 'temp_bid2 = %d'%(temp_bid)
                
                if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                #print order_book_bid[temp_bid]
                
                if position_ == 0 and len(order_book_bid[0]) > 1:
                    if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error110 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
    
                elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error220 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'  
                        continue
                elif position_ == len(order_book_bid[temp_bid])-1:
                        
                    if position_ > 0 and order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error330 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        continue
                    else:
                        print 'position and Best Price is True'                        
                        continue    
                elif position_ == 0 and len(order_book_bid[0]) == 1:
                    print 'position and Best Price is True'
                    continue                    
                                    
end = time.time()            
print "Total time = %f"%(end - start)    

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:22: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:23: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


------------------------------------------------------------------------------
12
False
position and Best Price is True
------------------------------------------------------------------------------
13
True
position and Best Price is True
------------------------------------------------------------------------------
14
True
position and Best Price is True
------------------------------------------------------------------------------
15
True
position and Best Price is True
------------------------------------------------------------------------------
16
position and Best Price is True
------------------------------------------------------------------------------
17
position and Best Price is True
------------------------------------------------------------------------------
18
position and Best Price is True
------------------------------------------------------------------------------
19
position and Best Price is True
-------------------------------------------------------------------

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:429: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


position and Best Price is True
------------------------------------------------------------------------------
199
position and Best Price is True
------------------------------------------------------------------------------
200
True
position and Best Price is True
------------------------------------------------------------------------------
201
False
position and Best Price is True
------------------------------------------------------------------------------
202
position and Best Price is True
------------------------------------------------------------------------------
203
position and Best Price is True
------------------------------------------------------------------------------
204
True
position and Best Price is True
------------------------------------------------------------------------------
205
False
position and Best Price is True
------------------------------------------------------------------------------
206
position and Best Price is True
--------------------------

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:380: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


position and Best Price is True
------------------------------------------------------------------------------
395
position and Best Price is True
------------------------------------------------------------------------------
396
position and Best Price is True
------------------------------------------------------------------------------
397
position and Best Price is True
------------------------------------------------------------------------------
398
position and Best Price is True
------------------------------------------------------------------------------
399
position and Best Price is True
------------------------------------------------------------------------------
400
position and Best Price is True
------------------------------------------------------------------------------
401
position and Best Price is True
------------------------------------------------------------------------------
402
True
position and Best Price is True
-------------------------------------------

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:211: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


position and Best Price is True
------------------------------------------------------------------------------
404
False
position and Best Price is True
------------------------------------------------------------------------------
405
position and Best Price is True
------------------------------------------------------------------------------
406
position and Best Price is True
------------------------------------------------------------------------------
407
True
position and Best Price is True
------------------------------------------------------------------------------
408
False
position and Best Price is True
------------------------------------------------------------------------------
409
position and Best Price is True
------------------------------------------------------------------------------
410
position and Best Price is True
------------------------------------------------------------------------------
411
position and Best Price is True
-------------------------------

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


position and Best Price is True
------------------------------------------------------------------------------
414
position and Best Price is True
------------------------------------------------------------------------------
415
position and Best Price is True
------------------------------------------------------------------------------
416
position and Best Price is True
------------------------------------------------------------------------------
417
position and Best Price is True
------------------------------------------------------------------------------
418
position and Best Price is True
------------------------------------------------------------------------------
419
False
position and Best Price is True
------------------------------------------------------------------------------
420
position and Best Price is True
------------------------------------------------------------------------------
421
position and Best Price is True
------------------------------------------

In [11]:
len(order_book_bid),len(data[0:3200][(data[0:3200].BidOrAsk == 'B')].TimeStamp.unique())

(1121, 1121)

In [12]:
len(order_book_ask),len(data[0:3200][(data[0:3200].BidOrAsk == 'A')].TimeStamp.unique())

(1157, 1157)

In [13]:
order_book_ask[len(order_book_ask)-1]

,Price,OrderNumber,QuantityDifference
0,674000,604A2D3D42A32343,10
1,674500,438A0CB584DF643F,10
2,674500,C9198CBD4D352CD5,10
3,674500,C48E292DA5DA867A,10
4,674500,CAC6296D8D816E21,10
5,674500,E0D6213503F5E496,5
6,674500,E0D22535403720D7,5
7,674500,490788F5610F44AF,6
8,674500,40C2A979626C42EC,10
9,674500,40CAA97990CB714B,10


In [14]:
order_book_bid[len(order_book_bid)-1]

,Price,OrderNumber,QuantityDifference
0,673500,E04E29353E751F15,2
1,673500,E0CE2935E067C107,10
2,673500,E05EA5654E0C2EAC,10
3,673500,E7DA0CE5C264A1C4,10
4,673500,67C42CA5A2308570,10
5,673500,401EAD79BA369AB6,10
6,673500,6FDC20A59AE67E26,10
7,673500,4752ADA1853E68BE,10
8,673500,EDCE00AD853A649A,10
9,673500,431D8CBD5B283EE8,13
